In [ ]:
import pandas as pd

file_path = "ukgranulargdhiothergeographies2002to2021.xlsx"
cities = ["York", "Belfast", "London", "Edinburgh"]
filtered_data = []

try:
    xls = pd.read_excel(file_path, sheet_name=None)
    
    for sheet_name, df in xls.items():
        print(f"Zpracovávám {sheet_name}...")
        
        # Check if the table has at least 4 columns
        if df.shape[1] < 4:
            print(f" {sheet_name} má méně než 4 sloupce, přeskočeno.")
            continue
        
        # 1st column for Table 13, 2nd column for others
        city_column = df.iloc[:, 0] if sheet_name == "Table 13" else df.iloc[:, 1]
        
        filtered_df = df[(city_column.isin(cities)) & (df.iloc[:, 3] == 'Gross Disposable Household Income')]
        
        if not filtered_df.empty:
            filtered_data.append(filtered_df)
    
    if filtered_data:
        final_df = pd.concat(filtered_data, ignore_index=True)

        # Remove the second occurrence of Edinburgh if its first column is empty
        final_df = final_df[~((final_df.iloc[:, 1] == "Edinburgh") & (final_df.iloc[:, 0].isna()))]

        # Uložíme výsledek
        final_df.to_excel("filtered_gdhi.xlsx", index=False)
        print("Filtered data has been saved as filtered_gdhi.xlsx")
    else:
        print("No data to save.")
    
except Exception as e:
    print(f"❌Error loading the file: {e}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def calculate_national_average_gdhi(file_path, city_gdhi_path):
    xls = pd.ExcelFile(file_path)
    
    df = pd.read_excel(xls, sheet_name='Table 1')
    
    df.columns = ["TTWA Code", "TTWA Name", "Transaction Code", "Transaction"] + list(range(2002, 2021 + 1))
    
    gdhi_national_df = df[df["Transaction"] == "Gross Disposable Household Income"]
    
    gdhi_national_df = gdhi_national_df.loc[:, 2002:2021]
    
    gdhi_national_df = gdhi_national_df.apply(pd.to_numeric, errors='coerce')
    
    national_avg_gdhi = gdhi_national_df.mean().to_frame().T
    national_avg_gdhi.insert(0, "City", "National Average")
    
    return national_avg_gdhi

def plot_gdhi_comparison(city_gdhi_path, national_avg_gdhi, output_dir="output_graphs"):
    os.makedirs(output_dir, exist_ok=True)

    data = pd.read_excel(city_gdhi_path)
    data_long = pd.melt(data, id_vars=['City'], var_name='Year', value_name='GDHI')
    data_long['Year'] = data_long['Year'].astype(int)
    data_long['Growth_Rate'] = data_long.groupby('City')['GDHI'].pct_change() * 100

    national_avg_long = pd.melt(national_avg_gdhi, id_vars=['City'], var_name='Year', value_name='GDHI')
    national_avg_long['Year'] = national_avg_long['Year'].astype(int)

    data_long = pd.concat([data_long, national_avg_long], ignore_index=True)

    # Plot and save graphs for individual cities
    cities = [c for c in data_long['City'].unique() if c != "National Average"]
    for city in cities:
        city_data = data_long[data_long['City'] == city]
        plt.figure(figsize=(10, 6))

        # GDHI graf
        plt.subplot(2, 1, 1)
        plt.plot(city_data['Year'], city_data['GDHI'], marker='o', label=city)
        plt.plot(national_avg_long['Year'], national_avg_long['GDHI'], marker='s', linestyle='--', label='National Average')
        plt.title(f'GDHI for {city}')
        plt.ylabel('GDHI (pounds million)')
        plt.grid(True)
        plt.xticks(city_data['Year'], rotation=45)
        plt.legend()

        # Growth rate graf
        plt.subplot(2, 1, 2)
        plt.plot(city_data['Year'], city_data['Growth_Rate'], marker='o', color='red', label=city)
        plt.plot(national_avg_long['Year'], national_avg_long['GDHI'].pct_change() * 100, marker='s', linestyle='--', color='blue', label='National Average')
        plt.title(f'Year-on-Year GDHI Growth for {city}')
        plt.xlabel('Year')
        plt.ylabel('Year-on-Year Growth (%)')
        plt.grid(True)
        plt.xticks(city_data['Year'], rotation=45)
        plt.legend()

        plt.tight_layout()

        
        save_path = os.path.join(output_dir, f'GDHI_Growth_{city}.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    # Comparison graph of all cities
    plt.figure(figsize=(12, 6))
    for city in cities:
        city_data = data_long[data_long['City'] == city]
        plt.plot(city_data['Year'], city_data['GDHI'], marker='o', label=city)

    plt.plot(national_avg_long['Year'], national_avg_long['GDHI'], marker='s', linestyle='--', color='purple', label='National Average')
    plt.title('Comparison of GDHI for Cities and National Average')
    plt.xlabel('Year')
    plt.ylabel('GDHI (pounds million)')
    plt.grid(True)
    plt.xticks(national_avg_long['Year'], rotation=45)
    plt.legend()

    save_path = os.path.join(output_dir, 'Comparison_All_Cities.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Graphs have been saved in the folder: {output_dir}")

file_path = "ukgranulargdhiothergeographies2002to2021.xlsx"
city_gdhi_path = "filtered_gdhi.xlsx"

national_avg_gdhi = calculate_national_average_gdhi(file_path, city_gdhi_path)
plot_gdhi_comparison(city_gdhi_path, national_avg_gdhi)


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

output_dir = "output_graphs"
os.makedirs(output_dir, exist_ok=True)

file_path = 'filtered_gdhi.xlsx'
data = pd.read_excel(file_path)

data_long = pd.melt(data, id_vars=['City'], var_name='Year', value_name='GDHI')
data_long['Year'] = data_long['Year'].astype(int)

# Add a new column for year-on-year growth
data_long['Growth_Rate'] = data_long.groupby('City')['GDHI'].pct_change() * 100

# Calculate the average annual growth rate for each city
average_growth_rates = data_long.groupby('City')['Growth_Rate'].mean().reset_index()

# Graph 1: Year-on-year growth comparison for all cities
plt.figure(figsize=(12, 8))

for city in data_long['City'].unique():
    city_data = data_long[data_long['City'] == city]
    plt.plot(city_data['Year'], city_data['Growth_Rate'], marker='o', label=city)

plt.title('Year-on-Year GDHI Growth Comparison for All Cities')
plt.xlabel('Year')
plt.ylabel('Year-on-Year Growth (%)')
plt.grid(True)
plt.legend()
plt.xticks(data_long['Year'].unique(), rotation=45)
plt.tight_layout()

save_path_1 = os.path.join(output_dir, 'Year-on-Year_GDHI_Growth_Comparison.png')
plt.savefig(save_path_1, dpi=300, bbox_inches='tight')
plt.close()

# Graph 2: Average annual growth rates for each city
plt.figure(figsize=(10, 6))
plt.bar(average_growth_rates['City'], average_growth_rates['Growth_Rate'], color='skyblue')
plt.title('Average Annual GDHI Growth Rate for Each City')
plt.xlabel('City')
plt.ylabel('Average Annual Growth Rate (%)')
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.tight_layout()

save_path_2 = os.path.join(output_dir, 'Average_Annual_GDHI_Growth_Rate.png')
plt.savefig(save_path_2, dpi=300, bbox_inches='tight')
plt.close()

print(f"Graphs have been saved in the folder:  {output_dir}")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore

def load_and_prepare_data(file_path):
    """Load the dataset and prepare it for analysis."""
    data = pd.read_excel(file_path)
    
    data_long = pd.melt(data, id_vars=['City'], var_name='Year', value_name='GDHI')
    data_long['Year'] = data_long['Year'].astype(int)
    
    data_long['Growth_Rate'] = data_long.groupby('City')['GDHI'].pct_change() * 100
    
    return data_long

def compute_correlation_and_volatility(data_long, selected_cities):
    """Compute correlation matrix and growth rate volatility."""
    # Compute correlation matrix for GDHI across the selected cities
    gdhi_pivot = data_long[data_long['City'].isin(selected_cities)].pivot(index='Year', columns='City', values='GDHI')
    correlation_matrix = gdhi_pivot.corr()
    
    # Compute standard deviation of growth rates to assess volatility
    growth_volatility = data_long[data_long['City'].isin(selected_cities)].groupby('City')['Growth_Rate'].std()
    
    return correlation_matrix, growth_volatility

def detect_anomalies(data_long):
    """Detect anomalies using Z-score method for growth rates."""
    data_long['Growth_Z_Score'] = data_long.groupby('City')['Growth_Rate'].transform(zscore)
    anomalies = data_long[data_long['Growth_Z_Score'].abs() > 2]
    return anomalies

file_path = "filtered_gdhi.xlsx"
selected_cities = ["York", "Belfast", "London", "Edinburgh"]

data_long = load_and_prepare_data(file_path)
correlation_matrix, growth_volatility = compute_correlation_and_volatility(data_long, selected_cities)
anomalies = detect_anomalies(data_long)

# Save results
correlation_matrix.to_excel("gdhi_correlation_matrix.xlsx")
growth_volatility.to_frame().to_excel("gdhi_growth_volatility.xlsx")
anomalies.to_excel("gdhi_anomalies.xlsx")

# Display results
print("Correlation Matrix:")
print(correlation_matrix)
print("\nGrowth Rate Volatility:")
print(growth_volatility)
print("\nAnomalies:")
print(anomalies)

In [ ]:
import pandas as pd

# Define coordinates for each city manually
city_coordinates = {
    "London": (51.5074, -0.1278),
    "York": (53.9590, -1.0815),
    "Belfast": (54.5973, -5.9301),
    "Edinburgh": (55.9533, -3.1883)
}

def export_kepler_data(file_path, output_path):
    data = pd.read_excel(file_path)
    
    data_long = pd.melt(data, id_vars=['City'], var_name='Year', value_name='GDHI')
    data_long['Year'] = data_long['Year'].astype(int)
    
    # Filter only the most recent available year (2021)
    data_latest = data_long[data_long['Year'] == 2021].copy()
    
    # Add coordinates
    data_latest.loc[:, 'Latitude'] = data_latest['City'].map(lambda city: city_coordinates[city][0])
    data_latest.loc[:, 'Longitude'] = data_latest['City'].map(lambda city: city_coordinates[city][1])
    
    # Save to CSV for Kepler.gl
    data_latest.to_csv(output_path, index=False)
    
    return data_latest

# Script usage
file_path = "filtered_gdhi.xlsx"
output_path = "gdhi_kepler_2021.csv"
data_for_kepler = export_kepler_data(file_path, output_path)

# Display the first few rows
print(data_for_kepler.head())


